In [2]:
import torch
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Subset
from tqdm import tqdm
import timm
from aimet_torch.batch_norm_fold import fold_all_batch_norms

/home/bmw/anaconda3/envs/swin_bhanu/lib/python3.10/site-packages/onnxscript/converter.py:823: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/home/bmw/anaconda3/envs/swin_bhanu/lib/python3.10/site-packages/onnxscript/converter.py:823: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()


2025-02-28 15:47:30,042 - root - INFO - AIMET


/home/bmw/anaconda3/envs/swin_bhanu/lib/python3.10/site-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/home/bmw/anaconda3/envs/swin_bhanu/lib/python3.10/site-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/home/bmw/anaconda3/envs/swin_bhanu/lib/python3.10/site-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/home/bmw/anaconda3/envs/swin_bhanu/lib/python3.10/site-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,


Process ForkProcess-8:
Process ForkProcess-12:
Process ForkProcess-9:
Process ForkProcess-10:
Process ForkProcess-11:
Process ForkProcess-1:
Process ForkProcess-7:
Process ForkProcess-3:
Process ForkProcess-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkProcess-6:
Traceback (most recent call last):
Process ForkProcess-5:
Process ForkProcess-16:
Traceback (most recent call last):
  File "/home/bmw/anaconda3/envs/swin_bhanu/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/bmw/anaconda3/envs/swin_bhanu/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/bmw/anaconda3/envs/swin_bhanu/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    

In [ ]:
device = torch.device("cpu")

model = timm.create_model("swin_tiny_patch4_window7_224", pretrained=True)

# Define transformation function
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

batch_size = 16
val_data_path = "/media/bmw/datasets/imagenet-1k/val/"
SUBSET_SIZE = 1000
val_dataset = torchvision.datasets.ImageFolder(root=val_data_path, transform=preprocess)

# Take a subset of the validation dataset
subset_indices = list(range(SUBSET_SIZE))  # First `SUBSET_SIZE` samples
val_subset = Subset(val_dataset, subset_indices)

# Create DataLoaders
val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, num_workers=4)

dummy_input = torch.randn(1, 3, 224, 224).to(device)
fold_all_batch_norms(model, dummy_input.shape)

# Callback function to pass calibration data through the model
def pass_calibration_data(model: torch.nn.Module, batches):
    with torch.no_grad():
        for batch, (images, _) in enumerate(val_loader):
            images = images.to(device)
            model(images)
            if batch >= batches:
                break

# Basic ImageNet evaluation function
def evaluate(model, data_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, labels in tqdm(data_loader):
            data, labels = data.to(device), labels.to(device)
            logits = model(data)
            correct += (logits.argmax(1) == labels).type(torch.float).sum().item()
    accuracy = correct / len(data_loader.dataset)
    return accuracy


2025-02-28 15:53:32,393 - timm.models.helpers - INFO - Loading pretrained weights from url (https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_tiny_patch4_window7_224.pth)
2025-02-28 15:53:33,381 - ConnectedGraph - WARNING - Unable to isolate model outputs.


In [5]:
from aimet_torch.model_preparer import prepare_model

print("\nPreparing and validating the model for quantization...")
prepared_model = prepare_model(model).to(device)


Preparing and validating the model for quantization...
2025-02-28 15:51:19,110 - ModelPreparer - INFO - Functional         : Adding new module for node: {layers.0.blocks.0.module_floordiv} 
2025-02-28 15:51:19,110 - ModelPreparer - INFO - Functional         : Adding new module for node: {layers.0.blocks.0.module_floordiv_1} 
2025-02-28 15:51:19,111 - ModelPreparer - INFO - Functional         : Adding new module for node: {layers.0.blocks.0.attn.module_floordiv_2} 
2025-02-28 15:51:19,111 - ModelPreparer - INFO - Functional         : Adding new module for node: {layers.0.blocks.0.attn.module_mul} 
2025-02-28 15:51:19,112 - ModelPreparer - INFO - Functional         : Adding new module for node: {layers.0.blocks.0.attn.module_matmul} 
2025-02-28 15:51:19,112 - ModelPreparer - INFO - Functional         : Adding new module for node: {layers.0.blocks.0.attn.module_add} 
2025-02-28 15:51:19,113 - ModelPreparer - INFO - Functional         : Adding new module for node: {layers.0.blocks.0.attn.

In [6]:
from aimet_torch.model_validator.model_validator import ModelValidator

ModelValidator.validate_model(prepared_model, model_input=torch.rand(1, 3, 224, 224).to(device))

2025-02-28 15:51:29,318 - Utils - INFO - Running validator check <function validate_for_reused_modules at 0x7efd864f2560>
2025-02-28 15:51:29,371 - Utils - INFO - Running validator check <function validate_for_missing_modules at 0x7efd7a4360e0>
2025-02-28 15:51:39,334 - ConnectedGraph - WARNING - Unable to isolate model outputs.
2025-02-28 15:51:39,359 - Utils - ERROR - Functional ops that are not marked as math invariant were found in the model. AIMET features will not work properly for such ops.
Consider the following choices: 
1. Redefine as a torch.nn.Module in the class definition.
2. The op can remain as a functional op due to being math invariant, but the op type has not been added to ConnectedGraph.math_invariant_types set. 
Add an entry to ignore the op by importing the set and adding the op type:

	from aimet_torch.meta.connectedgraph import ConnectedGraph
	ConnectedGraph.math_invariant_types.add(...)

The following functional ops were found. The parent module is named for ea

False

In [7]:
from aimet_torch.meta.connectedgraph import ConnectedGraph
ConnectedGraph.math_invariant_types.add("roll")

In [8]:
ModelValidator.validate_model(prepared_model, model_input=torch.rand(1, 3, 224, 224).to(device))

2025-02-28 15:52:27,345 - Utils - INFO - Running validator check <function validate_for_reused_modules at 0x7efd864f2560>
2025-02-28 15:52:27,397 - Utils - INFO - Running validator check <function validate_for_missing_modules at 0x7efd7a4360e0>
2025-02-28 15:52:37,069 - ConnectedGraph - WARNING - Unable to isolate model outputs.
2025-02-28 15:52:37,095 - Utils - INFO - All validation checks passed.


True

In [11]:
from aimet_common.defs import QuantScheme
from aimet_torch.quantsim import QuantizationSimModel
from timm.models.layers.drop import DropPath
from aimet_torch.v2.nn import QuantizationMixin


@QuantizationMixin.implements(DropPath)
class QuantizedDropPath(QuantizationMixin, DropPath):
    def __quant_init__(self):
        super().__quant_init__()
        self.input_quantizers = torch.nn.ModuleList([None])
        self.output_quantizers = torch.nn.ModuleList([None])

    def forward(self, x):
        if self.input_quantizers[0]:
            x = self.input_quantizers[0](x)
        with self._patch_quantized_parameters():
            ret = super().forward(x)
        if self.output_quantizers[0]:
            ret = self.output_quantizers[0](ret)
        return ret

    
sim = QuantizationSimModel(model, dummy_input, quant_scheme=QuantScheme.training_range_learning_with_tf_init)

calibration_batches = 10
sim.compute_encodings(pass_calibration_data, calibration_batches)

accuracy = evaluate(sim.model, val_loader)
print(f"Quantized accuracy (W8A8): {accuracy}")

2025-02-28 15:53:40,858 - ConnectedGraph - WARNING - Unable to isolate model outputs.
2025-02-28 15:53:41,046 - Quant - INFO - No config file provided, defaulting to config file at /home/bmw/anaconda3/envs/swin_bhanu/lib/python3.10/site-packages/aimet_common/quantsim_config/default_config.json
2025-02-28 15:53:41,068 - Quant - INFO - Unsupported op type Squeeze
2025-02-28 15:53:41,069 - Quant - INFO - Unsupported op type Mean
2025-02-28 15:53:41,084 - Quant - INFO - Selecting DefaultOpInstanceConfigGenerator to compute the specialized config. hw_version:default


100%|██████████| 63/63 [01:17<00:00,  1.23s/it]

Quantized accuracy (W8A8): 0.904


In [12]:
def train(model, data_loader, optimizer, loss_fn):
    model.train()
    for data, labels in tqdm(data_loader):
        data, labels = data.to(device), labels.to(device)
        logits = model(data)
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(sim.model.parameters(), lr=1e-5)

epochs = 2
for epoch in range(epochs):
    train(sim.model, val_loader, optimizer, loss_fn)

100%|██████████| 63/63 [05:17<00:00,  5.04s/it]


In [13]:
accuracy = evaluate(sim.model, val_loader)
print(f"Model accuracy after QAT: {accuracy}")

100%|██████████| 63/63 [01:17<00:00,  1.24s/it]

Model accuracy after QAT: 0.912
